In [1]:
import plotly.graph_objects as go
#!pip install plotly chart_studio --upgrade
import chart_studio.plotly as py
#라이브러리 설치방법은 다음과 같다.
# pip install cufflinks
# pip install chart_studio
import cufflinks as cf
cf.go_offline(connected=True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from textwrap import wrap


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS
from scipy.cluster.hierarchy import dendrogram

import re
import string
import time
from PIL import Image
from collections import Counter

import plotly.express as px

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as nltk_words
from nltk.corpus import stopwords
from matplotlib import font_manager, rc

rc('font', family='AppleGothic')

nltk.download('stopwords', download_dir='./resources')
nltk.download('punkt', download_dir='./resources')
nltk.download('wordnet', download_dir='./resources')
nltk.download('omw-1.4', download_dir='./resources')
nltk.download('words', download_dir='./resources')
nltk.data.path.append('./resources')

ModuleNotFoundError: No module named 'chart_studio'

사용 데이터 <br>
https://www.kaggle.com/datasets/hadasu92/cnn-articles-after-basic-cleaning [CNNNews]

In [19]:
data = pd.read_csv('data/CNN_뉴스.csv', encoding='CP949')
data

/var/folders/w7/wdh3k6ns0dsgm7bmq_lzxw0h0000gn/T/ipykernel_35088/3123962252.py:1: DtypeWarning:

Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.



,title,pubDate,guid,link,description,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 739,Unnamed: 740,Unnamed: 741,Unnamed: 742,Unnamed: 743,Unnamed: 744,Unnamed: 745,Unnamed: 746,Unnamed: 747,Unnamed: 748
0,"Two years later, remote work has changed milli...",2022.3.18 14:37,https://www.cnn.com/2022/03/18/success/pandemi...,https://www.cnn.com/2022/03/18/success/pandemi...,Here's a look at how the pandemic reshaped peo...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Why March is so volatile for stocks - CNN,2022.3.19 11:41,https://www.cnn.com/2022/03/19/investing/march...,https://www.cnn.com/2022/03/19/investing/march...,March Madness isn't just for college basketbal...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stocks week ahead: Big Oil rakes in billions a...,2022.3.20 11:36,https://www.cnn.com/2022/03/20/investing/stock...,https://www.cnn.com/2022/03/20/investing/stock...,"As crude prices surge, oil companies are rakin...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oil 'emergency': Work from home and drive slow...,2022.3.18 14:26,https://www.cnn.com/2022/03/18/energy/oil-russ...,https://www.cnn.com/2022/03/18/energy/oil-russ...,Governments around the world must consider dra...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Opinion: Technology is transforming the nature...,2022.3.20 12:57,https://www.cnn.com/2022/03/20/perspectives/im...,https://www.cnn.com/2022/03/20/perspectives/im...,The convenience of digital payments to both co...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,Vladimir Putin faces stiffer opposition than e...,2022.2.28 13:30,https://www.cnn.com/2022/02/28/europe/putin-st...,https://www.cnn.com/2022/02/28/europe/putin-st...,"Five days into Russia's invasion of Ukraine, i...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3302,"Ja Morant has historic night, breaks own scori...",2022.3.1 11:43,https://www.cnn.com/2022/03/01/sport/ja-morant...,https://www.cnn.com/2022/03/01/sport/ja-morant...,What can't Ja Morant do?,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3303,Russian President Vladimir Putin is being stri...,2022.3.1 10:59,https://www.cnn.com/2022/03/01/sport/vladimir-...,https://www.cnn.com/2022/03/01/sport/vladimir-...,Russian President Vladimir Putin has been stri...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3304,"On NATO's doorstep, a former tourist hotspot i...",2022.3.1 12:55,https://www.cnn.com/2022/03/01/europe/nato-ukr...,https://www.cnn.com/2022/03/01/europe/nato-ukr...,A long line of men snakes out of an unassuming...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
sns.set_style('darkgrid')

In [21]:
STOPWORDS = stopwords.words('english')
pd.Series(STOPWORDS)

0             i
1            me
2            my
3        myself
4            we
         ...   
174     weren't
175         won
176       won't
177      wouldn
178    wouldn't
Length: 179, dtype: object

In [22]:
# BBC 언급 단어 Top15 추출
top_N = 15
a = data['title'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.cat(sep=' ')
words = list(filter(lambda w: w not in STOPWORDS, nltk.tokenize.word_tokenize(a)))
word_dist = nltk.FreqDist(words)
freq_title = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])

# 대부분의 title에 CNN, cnnpolitics가 포함되어 cnn제거
freq_title.drop(0, inplace=True)
freq_title.drop(1, inplace=True)

freq_title

,Word,Frequency
2,ukraine,261
3,says,242
4,us,233
5,new,176
6,russia,164
7,biden,152
8,russian,147
9,opinion,144
10,covid19,111
11,2022,96


In [23]:
freq_title.set_index('Word',inplace=True)

In [27]:
# BBC 언급 단어 Top15 그래프
freq_title[::-1].iplot(kind='bar', orientation="h", title='CNN 최다 언급 단어 순위')